# Analyzing borrowers’ risk of defaulting

This project is designed to analyze the risk of borrowers defaulting on a loan on the basis of their marital status and the number of children. 

In [1]:
# Loading all the libraries
import pandas as pd
from nltk.stem import SnowballStemmer
# Load the data
try:
    df = pd.read_csv('E:\\Users\\x\\Documents\\jupyter\\DP\\credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

**Descriptions**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [2]:
# Quantity of rows and columns
df.shape

(21525, 12)

In [3]:
# First 10 rows
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [4]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
# Missing values on dataset
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# Calculate percentage of missing values contained in the datasets.
df['days_employed'].isna().sum()/len(df)*100

10.099883855981417

In [7]:
# Checking distribution 
df.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,19351.000000,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,wedding ceremony
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,26787.568355,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,16475.450632,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,3306.762000,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,16488.504500,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,23202.870000,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,32549.611000,NaN


In [8]:
# Verify if the missing values in column days_employed and total_income are exactly the same
null_value = df[df.isnull().any(axis=1)]
null_value[['days_employed','total_income']].head()

,days_employed,total_income
12,NaN,NaN
26,NaN,NaN
29,NaN,NaN
41,NaN,NaN
55,NaN,NaN


**Initialization**
  
- 2174 missing values on both `days_employed` and `total_income` columns. 
- `days_employed` data showed negative values and should be better represented with integer data type instead. 
- `total_income` datatype could be represented in integer as well or values without decimals. 

Since missing values seems significant. It is approximately 10% of the whole data set. 
Possible wrong data entered on `days_employed` column and customer might choose to not share their `total_income` data due to privacy concern during the data collection. 
The missing values on `days_employed` column matched exactly to `total_income` column. Customers who are unemployed are not comfortable sharing their income information. 

## Data transformation

In [9]:
# Verify all the values in education columns
df['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

In [10]:
# Make data lower case
df['education']=df['education'].str.lower()

In [11]:
# Recheck all the values in the column
df['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [12]:
# Children's column
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

- Possible error on counts with 20 on `children`.
- Negative value on `children` seems incorrect as well

In [13]:
# percent of children with -1 value in data sets
print(len(df[df['children']<0])/(df['children'].count())*100)

0.2183507549361208


In [14]:
# percent of 20 children within the data sets
print(len(df[df['children']==20])/(df['children'].count())*100)

0.3530778164924506


In [15]:
# replace negative value (-1) to 1
df.loc[df['children']<0,'children']=1

In [16]:
# dropping rows with value of 20 children
df.drop(index=df[df['children']==20].index,inplace=True)

In [17]:
# Checking the `children` column again to make sure it's all fixed
df['children'].value_counts()

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Data errors of `children` seem very small which are only 0.2% and 0.35% of the datasets. The errors had been fixed by replacing the -1 to 1 and dropping the entire row of data with `children == 20` 

In [18]:
# Find missing values days_employed
df['days_employed'].isna().value_counts()

False    19284
True      2165
Name: days_employed, dtype: int64

In [19]:
# Address the problematic values, if they exist
df['days_employed'].isna().sum()/len(df)*100

10.093710662501747

In [20]:
#check unique value of dob_years
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [21]:
# Check the `dob_years` for suspicious values and count the percentage
(len(df[df['dob_years']==0].count()))/len(df)*100

0.055946664180148264

In [22]:
# Address the issues in the `dob_years` column, if they exist
df.drop(index=df[df['dob_years']<1].index,inplace=True)

In [23]:
# Check the result - make sure it's fixed
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

Age 0 seems like an error in the dataset. Dropping those rows won't affect the dataframe much because the error only represent approximately 0.05% of the whole dataframe. Dropping row of `dob_years` with 0.

In [24]:
# Family_status values
df['family_status'].value_counts()

married              12283
civil partnership     4144
unmarried             2788
divorced              1183
widow / widower        951
Name: family_status, dtype: int64

`Unmarried` sounds similar to married, replacing with `single` for better and easier interpretation.

In [25]:
# Replace unmarried to single
df['family_status']=df['family_status'].replace('unmarried','single')

In [26]:
# Check the result - make sure it's fixed
df['family_status'].value_counts()

married              12283
civil partnership     4144
single                2788
divorced              1183
widow / widower        951
Name: family_status, dtype: int64

In [27]:
# Gender values
df['gender'].value_counts()

F      14118
M       7230
XNA        1
Name: gender, dtype: int64

In [28]:
#replace XNA gender with common gender
df['gender']=df['gender'].replace('XNA','F')

In [29]:
# Check the result - make sure it's fixed
df['gender'].value_counts()

F    14119
M     7230
Name: gender, dtype: int64

`XNA` data in `gender` column seems like an unknown value. Luckily, there is only one XNA value in the gender column, replacing it with the more common gender from the dataset which is `F`. 

In [30]:
# Let's see the values in the column
df['income_type'].value_counts()

employee                       11022
business                        5043
retiree                         3827
civil servant                   1451
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [31]:
# Checking duplicates on whole datasets
df.duplicated().value_counts()

False    21278
True        71
dtype: int64

In [32]:
# Address the duplicates, if they exist
df = df.drop_duplicates().reset_index(drop=True)

In [33]:
# Last check whether we have any duplicates
df.duplicated().value_counts()

False    21278
dtype: int64

In [34]:
# Check the size of the dataset that you now have after your first manipulations with it
df.shape

(21278, 12)

There were originally 21525 rows of data, all columns now down to 21278 rows after data handling.

# Working with missing values

In [35]:
# Counts of missing value in total_income
df['total_income'].isna().value_counts()

False    19194
True      2084
Name: total_income, dtype: int64

### Restoring missing values in `total_income`

In [36]:
# Describe total_income column
df['total_income'].describe()

count     19194.000000
mean      26794.003801
std       16504.921426
min        3306.762000
25%       16493.600500
50%       23203.328000
75%       32535.277250
max      362496.645000
Name: total_income, dtype: float64

In [37]:
# Create a table without missing values
income_data = df[df['total_income'].isnull()==False]
income_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [38]:
# Look at the mean and median values for income based on your identified factors
income_data['total_income'].describe()

count     19194.000000
mean      26794.003801
std       16504.921426
min        3306.762000
25%       16493.600500
50%       23203.328000
75%       32535.277250
max      362496.645000
Name: total_income, dtype: float64

In [39]:
# Group income_data
def df_income_group(income):
    if income < 33000:
        return 'low income'
    elif income < 120000:
        return 'middle income'
    else:
        return 'upper income'
# Test function
income_data['total_income'].apply(df_income_group).value_counts()

low income       14569
middle income     4576
upper income        49
Name: total_income, dtype: int64

In [40]:
# apply to new column 
df['group_income']=income_data['total_income'].apply(df_income_group)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_income
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,middle income
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,low income
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,low income
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,middle income
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,low income


In [41]:
# Look at the distribution for income based on income_type
df.groupby('income_type')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
business,4540.0,32414.336508,20936.640782,4592.450,20111.84275,27583.3600,39028.97350,362496.645
civil servant,1305.0,27350.649756,15525.907930,4672.012,16814.86800,24076.1150,33452.06600,145672.235
employee,9925.0,25819.737239,14626.392529,3418.824,16447.20300,22816.1930,31468.05400,276204.162
entrepreneur,1.0,79866.103000,NaN,79866.103,79866.10300,79866.1030,79866.10300,79866.103
paternity / maternity leave,1.0,8612.661000,NaN,8612.661,8612.66100,8612.6610,8612.66100,8612.661
retiree,3419.0,21943.056865,12846.672456,3306.762,13271.68350,18956.9340,27134.30750,117616.523
student,1.0,15712.260000,NaN,15712.260,15712.26000,15712.2600,15712.26000,15712.260
unemployed,2.0,21014.360500,16152.074628,9593.119,15303.73975,21014.3605,26724.98125,32435.602


In [42]:
# Look at the distribution for income based on education
df.groupby('education')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
bachelor's degree,4673.0,33186.447659,21767.638121,5148.514,20280.74500,28065.7400,40050.2770,362496.645
graduate degree,6.0,27960.024667,12205.330046,15800.399,18005.02925,25161.5835,38593.8535,42945.794
primary education,261.0,21144.882211,10873.977874,4049.374,13117.13300,18741.9760,27119.0240,78410.774
secondary education,13583.0,24590.586912,13702.454949,3306.762,15621.68800,21835.2490,30212.2745,276204.162
some college,671.0,29066.054548,15662.897367,5514.581,18225.26950,25709.8980,36628.2880,153349.533


In [43]:
#  Replace missing values in total_income with median grouped by education
df['total_income'] = df['total_income'].fillna(df.groupby('education')['total_income'].transform('median'))

In [44]:
# Check if we got any errors
df['total_income'].isna().value_counts()

False    21278
Name: total_income, dtype: int64

In [45]:
# Convert total_income datatype from float to int
df['total_income']=df['total_income'].astype('int64')

In [46]:
# apply total_income to group.
df['group_income']=df['total_income'].apply(df_income_group)

In [47]:
# Group_income's value
df['group_income'].value_counts()

low income       16653
middle income     4576
upper income        49
Name: group_income, dtype: int64

In [48]:
# Checking the number of entries in the columns
df['total_income'].size

21278

In [49]:
# Let's write a function that calculates the age category
def df_age_group(age):
    if age < 30:
        return '20s'
    elif age < 40:
        return '30s'
    elif age < 50:
        return '40s'
    elif age < 60:
        return '50s'
    elif age < 70:
        return '60s'
    else: 
        return '70+'

In [50]:
# Test if the function works
df['dob_years'].apply(df_age_group).value_counts()

30s    5640
40s    5331
50s    4641
20s    3171
60s    2326
70+     169
Name: dob_years, dtype: int64

In [51]:
# Creating new column based on function
df['age_group']=df['dob_years'].apply(df_age_group)

In [52]:
# Checking how values in the new column
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_income,age_group
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,middle income,40s
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932,car purchase,low income,30s
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,low income,30s
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,middle income,30s
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,low income,50s
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922,purchase of the house,middle income,20s
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484,housing transactions,middle income,40s
7,0,-152.779569,50,secondary education,1,married,0,M,employee,0,21731,education,low income,50s
8,2,-6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337,having a wedding,low income,30s
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108,purchase of the house for my family,low income,40s


In [53]:
df.groupby('age_group')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
age_group,,,,,,,,
20s,3171.0,25404.631347,12893.230966,4494.0,16906.5,22493.0,29815.50,131588.0
30s,5640.0,27875.110638,16638.312238,3392.0,18393.0,24063.5,32786.00,352136.0
40s,5331.0,28063.781842,17173.288778,4036.0,18203.0,23997.0,33458.00,362496.0
50s,4641.0,25529.527042,14962.731505,3306.0,16446.0,21835.0,30065.00,195686.0
60s,2326.0,23210.030954,14459.133488,3471.0,14508.5,21340.5,27781.75,274402.0
70+,169.0,20234.781065,9765.565653,3595.0,12944.0,19163.0,25233.00,57508.0


- Original plan was replacing those *missing values* with the *median()* obtained from total_income itself, but the data distribution is too skewed even for *median()* values.  

- Replaced those missing values with *median()* values grouped by `education` will results overall better datasets. 

- Added `Group_income` for better interpretation of datasets as well. Grouped into: 

    - low income 

    - middle income 

    - upper income 

- Added `Age_group` in to 30s to 70+ category for better interpretation as well. 

  

Datasets showed most borrowers are in the low-income group. 

###  Restoring values in `days_employed`

In [54]:
# Look at days_employed column
df['days_employed'].head(10)

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

In [55]:
# remove negative value of day_employed column
df['days_employed']=abs(df['days_employed'])
# check column after remove
df['days_employed'].head(10)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

In [56]:
# `days_employed` medians and means
df['days_employed'].describe()

count     19194.000000
mean      66994.286518
std      139101.094784
min          24.141633
25%         927.830608
50%        2198.029232
75%        5554.325716
max      401755.400475
Name: days_employed, dtype: float64

In [57]:
# `days_employed` medians and means grouped by 'education'
df.groupby('education')['days_employed'].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
bachelor's degree,4673.0,42447.427842,114140.223551,24.240695,819.627219,1896.866606,4128.038397,401715.811749
graduate degree,6.0,121323.630206,182921.141764,409.200149,3101.583496,5660.057032,254680.630641,376276.219531
primary education,261.0,130340.426349,173352.954149,62.108487,863.272579,3043.933615,346471.063425,401440.834336
secondary education,13583.0,76481.124303,146492.386656,24.141633,997.081407,2394.069195,7082.504942,401755.400475
some college,671.0,20777.348241,82078.444889,51.496885,598.088924,1212.588325,2259.270736,399693.485689


In [58]:
# `days_employed` medians and means grouped by 'income_type'
df.groupby('income_type')['days_employed'].describe()

,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
business,4540.0,2117.933008,2052.113581,30.195337,686.382042,1555.261159,2882.671797,17615.563266
civil servant,1305.0,3387.514221,2784.226064,39.954170,1246.749979,2672.903939,4726.161617,15193.032201
employee,9925.0,2327.164891,2308.561853,24.141633,745.352610,1575.323578,3108.834443,18388.949901
entrepreneur,1.0,520.848083,NaN,520.848083,520.848083,520.848083,520.848083,520.848083
paternity / maternity leave,1.0,3296.759962,NaN,3296.759962,3296.759962,3296.759962,3296.759962,3296.759962
retiree,3419.0,365024.240512,21064.054513,328728.720605,346697.593443,365247.114512,383265.981921,401755.400475
student,1.0,578.751554,NaN,578.751554,578.751554,578.751554,578.751554,578.751554
unemployed,2.0,366413.652744,40855.478519,337524.466835,351969.059790,366413.652744,380858.245699,395302.838654


In [59]:
# Replacing missing values with median group by income_type
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))

In [60]:
# Convert days_employed datatype from float to int
df['days_employed']=df['days_employed'].astype('int64')

In [61]:
# Checking if missing values are all filled
df['days_employed'].isna().value_counts()

False    21278
Name: days_employed, dtype: int64

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21278 entries, 0 to 21277
Data columns (total 14 columns):
children            21278 non-null int64
days_employed       21278 non-null int64
dob_years           21278 non-null int64
education           21278 non-null object
education_id        21278 non-null int64
family_status       21278 non-null object
family_status_id    21278 non-null int64
gender              21278 non-null object
income_type         21278 non-null object
debt                21278 non-null int64
total_income        21278 non-null int64
purpose             21278 non-null object
group_income        21278 non-null object
age_group           21278 non-null object
dtypes: int64(7), object(7)
memory usage: 2.3+ MB


- Distribution of data in `days_employed` are too skewed to be used to replace its missing values. 

- Replaced those missing values with *median()* values grouped by `income_type` probably produced better datasets because they are existing data. 

## Categorization of data

In [63]:
# Print the values for your selected data for categorization
df['purpose'].head(15)

0                   purchase of the house
1                            car purchase
2                   purchase of the house
3                 supplementary education
4                       to have a wedding
5                   purchase of the house
6                    housing transactions
7                               education
8                        having a wedding
9     purchase of the house for my family
10                        buy real estate
11             buy commercial real estate
12                      to have a wedding
13                           car purchase
14            buy residential real estate
Name: purpose, dtype: object

In [64]:
# Check the unique values
df['purpose'].value_counts()

wedding ceremony                            785
having a wedding                            760
to have a wedding                           756
real estate transactions                    671
buy commercial real estate                  655
buying property for renting out             648
transactions with commercial real estate    644
housing transactions                        642
purchase of the house                       637
housing                                     636
purchase of the house for my family         636
property                                    628
construction of own property                627
transactions with my real estate            626
building a real estate                      620
building a property                         619
purchase of my own house                    619
buy real estate                             616
housing renovation                          603
buy residential real estate                 601
buying my own car                       

In [65]:
# Function to categorize the purpose data
english_stemmer = SnowballStemmer('english')
def categorizer(issue):
    if issue == "profile education":
        return "education"
    elif issue == "university education":
        return "education"
    elif issue == "supplementary education":
        return "education"
    for word in issue.replace(',',' ').replace('/',' ').split(" "):
        stem = english_stemmer.stem(word)
        if stem == "wed":
            return "wedding"
        elif stem == "build" or stem == "hous" or stem == "rent" or stem == "comm"or stem == "real" or stem=="esta"or stem == "own":
            return "property"
        elif stem == "car":
            return "automotive"
        elif stem == "educated" or stem == "higher"or stem == "an" or stem == "to" or stem == "university" or stem == "profile":
            return "education"
    return issue

In [66]:
# Create a column with the categories and count the values for them
df['cat_purpose']=df['purpose'].apply(categorizer)

In [67]:
# Looking through all the numerical data in your selected column for categorization
df['cat_purpose'].value_counts()

property      11230
education      5674
automotive     2829
wedding        1545
Name: cat_purpose, dtype: int64

In [68]:
# Getting summary statistics for the column
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_income,age_group,cat_purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,middle income,40s,property
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,low income,30s,automotive
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,low income,30s,property
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,middle income,30s,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,low income,50s,education
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,0,40922,purchase of the house,middle income,20s,property
6,0,2879,43,bachelor's degree,0,married,0,F,business,0,38484,housing transactions,middle income,40s,property
7,0,152,50,secondary education,1,married,0,M,employee,0,21731,education,low income,50s,education
8,2,6929,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337,having a wedding,low income,30s,wedding
9,0,2188,41,secondary education,1,married,0,M,employee,0,23108,purchase of the house for my family,low income,40s,property


There are 38 unique values in `purpose` columns. Categorized them into 4 categories make it much easier to read data: 

   - property 

   - education 

   - automotive 

   - wedding 
 

## Checking the Hypotheses


In [69]:
df['debt'].value_counts()

0    19553
1     1725
Name: debt, dtype: int64

In [70]:
# Check the children data and paying back on time
default_children=df.pivot_table(index='children',columns='group_income',values='debt',aggfunc='sum',margins='True')
# Calculating default-rate based on the number of children
default_children

group_income,low income,middle income,upper income,All
children,,,,
0,861.0,194.0,3.0,1058
1,361.0,80.0,1.0,442
2,156.0,38.0,0.0,194
3,23.0,4.0,0.0,27
4,3.0,1.0,NaN,4
5,0.0,0.0,NaN,0
All,1404.0,317.0,4.0,1725


In [71]:
# Check the family status data and paying back on time
default_family=df.pivot_table(index='family_status',columns='group_income',values='debt',aggfunc='sum',margins='True')
# Calculating default-rate based on family status
default_family

group_income,low income,middle income,upper income,All
family_status,,,,
civil partnership,324.0,58.0,1.0,383
divorced,68.0,16.0,0.0,84
married,742.0,179.0,3.0,924
single,216.0,56.0,0.0,272
widow / widower,54.0,8.0,NaN,62
All,1404.0,317.0,4.0,1725


In [72]:
# Check the income level data and paying back on time
default_income=df.pivot_table(index='group_income',columns='income_type',values='debt',aggfunc='sum',margins='True')
# Calculating default-rate based on income level
default_income

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student,unemployed,All
group_income,,,,,,,,,
low income,272.0,71.0,872.0,0.0,1.0,187.0,0.0,1.0,1404
middle income,98.0,15.0,176.0,0.0,NaN,28.0,NaN,NaN,317
upper income,3.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,4
All,373.0,86.0,1049.0,0.0,1.0,215.0,0.0,1.0,1725


In [73]:
# Check the percentages for default rate for each credit purpose for analysis
default_children/len(df[df['debt']==1])*100

group_income,low income,middle income,upper income,All
children,,,,
0,49.913043,11.246377,0.173913,61.333333
1,20.927536,4.637681,0.057971,25.623188
2,9.043478,2.202899,0.000000,11.246377
3,1.333333,0.231884,0.000000,1.565217
4,0.173913,0.057971,NaN,0.231884
5,0.000000,0.000000,NaN,0.000000
All,81.391304,18.376812,0.231884,100.000000


In [74]:
default_family/len(df[df['debt']==1])*100

group_income,low income,middle income,upper income,All
family_status,,,,
civil partnership,18.782609,3.362319,0.057971,22.202899
divorced,3.942029,0.927536,0.000000,4.869565
married,43.014493,10.376812,0.173913,53.565217
single,12.521739,3.246377,0.000000,15.768116
widow / widower,3.130435,0.463768,NaN,3.594203
All,81.391304,18.376812,0.231884,100.000000


In [75]:
default_income/len(df[df['debt']==1])*100

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student,unemployed,All
group_income,,,,,,,,,
low income,15.768116,4.115942,50.550725,0.0,0.057971,10.840580,0.0,0.057971,81.391304
middle income,5.681159,0.869565,10.202899,0.0,NaN,1.623188,NaN,NaN,18.376812
upper income,0.173913,0.000000,0.057971,NaN,NaN,NaN,NaN,NaN,0.231884
All,21.623188,4.985507,60.811594,0.0,0.057971,12.463768,0.0,0.057971,100.000000


# Conclusion  

  

**Children vs Debt** 

- Having children does not seem to have much impact on debt repayment on time. 

- Most of the low income without any children have the highest debt default rate. Which is approximately 49% of the datasets `children-debt`. 

- Having more children and low incomes has a general impact on debt repayment on time. 

- Number of children might not be the main effect on paying debt back on time. 

  

**Family Status vs Debt** 

- Married couples with low income have the highest rate of debt default. 

- Divorced and widow/widower have less impact on debt default. 

- Married couples have a significant impact on paying debt on time. This is a total of about 54% of the family's overall status in all income brackets to affect debt. 

  

**Income_level vs Debt** 

- Low-income households have shown to have a significant impact on debt default. 

- Approximately 81% of the debt default falls on low-income group. 

- On the opposite side, upper income group have hardly any debt default in datasets at all. (Only 0.2% of the total datasets) 

  

Based on the analysis from datasets with the provided data, *low income* household have significantly higher chance to default on debt. Surprisingly, data showed that households without children had a higher rate of debt default, which usually was not the case because households without children might have less commitment and be less likely to be in financial hardship. So, having kids will not increase the rate of debt default.  

  

- ***Borrower with conditions below able to reduce the debt default risk:*** 

- `income status of middle to upper income` 

- `family status divorce, widower, single` 

- `number of children > 1. ` 